<a href="https://colab.research.google.com/github/hugegene/LSTM-Prediction-of-Stock-Price-Movement/blob/master/CombinedSentimentTestingStockLoopForMarketPsych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!kill -9 -1

## 3.1Upload Data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving AmazonPrices.csv to AmazonPrices.csv
Saving AmazonSentiments.csv to AmazonSentiments.csv
Saving AMDPrices.csv to AMDPrices.csv
Saving AMDSentiments.csv to AMDSentiments.csv
Saving ApplePrices.csv to ApplePrices.csv
Saving AppleSentiments.csv to AppleSentiments.csv
Saving FacebookPrices.csv to FacebookPrices.csv
Saving FacebookSentiments.csv to FacebookSentiments.csv
Saving GoogPrices.csv to GoogPrices.csv
Saving GoogSentiments.csv to GoogSentiments.csv
Saving MicronPrices.csv to MicronPrices.csv
Saving MicronSentiments.csv to MicronSentiments.csv
Saving MicrosoftPrices.csv to MicrosoftPrices.csv
Saving MicrosoftSentiments.csv to MicrosoftSentiments.csv
Saving NetflixPrices.csv to NetflixPrices.csv
Saving NetflixSentiments.csv to NetflixSentiments.csv
Saving NvidiaPrices.csv to NvidiaPrices.csv
Saving NvidiaSentiments.csv to NvidiaSentiments.csv
Saving TeslaPrices.csv to TeslaPrices.csv
Saving TeslaSentiments.csv to TeslaSentiments.csv


In [0]:
!ls

In [0]:
!rm roll8ENS3Train488TEST3ACC0.51AUC0.5MCC-0.0TRMI.csv

#  Data Pre-Processing

In [0]:
#For seeding when unit testing, do not seed for the experiment
# from numpy.random import seed
# seed(1)
# from tensorflow import set_random_seed
# set_random_seed(2)
# from random import seed
# seed(3)

import keras
from keras.utils import np_utils
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import model_from_json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from datetime import datetime
import os
import itertools
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from keras.layers import Input, LSTM, RepeatVector, Lambda, Dense, Flatten, Permute, merge, multiply, Dropout
from keras.layers import Dense, LSTM, GlobalMaxPooling1D
from keras.models import Model
from keras import backend as K
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score as areauc
import tensorflow as tf
from datetime import timedelta, date
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', 600)
np.set_printoptions(threshold=np.nan)
from matplotlib.pyplot import cm
from sklearn.utils import class_weight
from keras.layers.wrappers import Bidirectional
from keras.layers.normalization import BatchNormalization
from google.colab import files
import time

#Define function to convert dataframe to training, validating and testing examples
def series_to_supervised(df, n_in=1, n_out=1):
	n_vars = 1 if type(df) is list else df.shape[1]
  
  
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, -1, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]

	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	return agg

In [0]:
# list of Stocks
read = ['Amazon', 'Apple', 'Facebook', 'Tesla', 'Netflix', 'Microsoft', 'Micron', 'Nvidia', 'Goog', 'AMD']

# Setup empty dataframe
Sets = pd.DataFrame()

for r in range(len(read)):
  ##Read in Price data, use date as index
  filename = read[r]
  print(filename)
  Prices = pd.read_csv(filename+ "Prices.csv")
  Prices.head()
  Prices['Date'].dtype
  Prices['Date'] = pd.to_datetime(Prices['Date'], format='%Y-%m-%d')
  Prices.head()
  Prices.index = Prices['Date']
  # print(Prices.head())

  #Reading in Sentiment Data and selecting News-only-MarketRisk Sentiment
  Sent = pd.read_csv(filename + "Sentiments.csv")
  Sent = Sent[Sent.dataType=='News'][['Date', 'buzz', 'sentiment', 'optimism', 'joy', 'loveHate',
         'trust', 'anger', 'conflict', 'fear', 'gloom', 'stress', 'surprise',
         'timeUrgency', 'uncertainty', 'violence', 'emotionVsFact', 'marketRisk',
         'longShort', 'longShortForecast', 'priceDirection', 'priceForecast',
         'volatility', 'analystRating', 'debtDefault', 'dividends', 'innovation',
         'earningsForecast', 'fundamentalStrength', 'laborDispute', 'layoffs',
         'litigation', 'managementChange', 'managementTrust', 'mergers',
         'cyberCrime']].fillna(method = "ffill")

  Sent['Date'] = pd.to_datetime(Sent['Date'], format='%Y-%m-%d')
  Sent.index = Sent['Date']
  Sent.fillna(0, axis=1, inplace = True)
  # print(Sent.head())

  #Toggle TRMI to use TRMI or not TRMI to not use TRMI
#   TRMI ="NOTRMI"
  TRMI = "TRMI"
  print(TRMI)
  #Combining Price table and Sentiment Table
  if TRMI == "TRMI":
    print("TRMIIIIIIIIIIIII")
    Ana = Prices.merge(Sent, left_index= True, right_index=True, how='inner')
    Ana = Ana.drop(columns=['Unnamed: 0', 'Date_x', 'Date_y', 'Volume', 'Asset', 'Open', 'High', 'Low', 'UnadjClose'])
  else:
    Ana= Prices.copy()
    Ana = Ana.drop(columns=['Unnamed: 0', 'Date', 'Volume', 'Asset', 'Open', 'High', 'Low', 'UnadjClose'])

  Ana = Ana["2013-01-01":]
  Ana.head()


  #Feature Engineering
  for i in Ana.columns:
    if i == "Close":
      Ana["r1"] =  Ana["Close"].diff(1)
      Ana["r2"] =  Ana["Close"].diff(7)
      Ana["r3"] =  Ana["Close"].diff(30)
      Ana["r4"] =  Ana["Close"].diff(90)
  
    else:
      avgs = [7, 30, 90]
      for j in range(len(avgs)-1):
        varrName =  i +"_diff" + str(avgs[j])
        varName =  i +"_avg" + str(avgs[j]) + "_diff" + str(avgs[j+1]) + "_avg"
        Ana[varName] = Ana[i].rolling(avgs[j]).mean() - Ana[i].rolling(avgs[j+1]).mean()
#         Ana[varrName] = Ana[i].diff(j)

#   Ana.head()
#   print(Ana.columns)

#   calculating r, %tage change of closing price over previous time step
  Ana['returns'] = (Ana['Close']- Ana['Close'].shift(1))/Ana['Close'].shift(1)
  Ana.head()

  # calculating target price
  Ana['Target'] = Ana["Close"].diff(1).shift(-1)
  Ana.head()

  # calculating %change
  Ana['Change'] = Ana["returns"].shift(-1)
  Ana

  # Label each time step False(no-buy) or True(buy) based on whether the price will rise at the next closing price
  timestep = 1
  labels = []
  for i in range(0, Ana.shape[0]):
    if(i+timestep< Ana.shape[0]):
      aheadGain = [Ana["returns"][i+j] for j in range(1,timestep+1)]
      labels+= [np.sum(aheadGain)> 0]

  # print(len(labels))

  # Price Table with Labels as Signal
  Ana = Ana.iloc[:-timestep,:].copy()
  Ana.head()
  Ana['Signal'] = labels
  Ana.head()

  #Drop NA
  Ana = Ana.dropna(0)

  #Features Selection
  Set =  Ana[Ana.columns[1:-4]]
  Set["AssetNo"] = r
  done = ["0","1","2","3","4","5","6","7","8","9"]
  dummies = pd.get_dummies(Set["AssetNo"], prefix='', prefix_sep='')
  dummies = dummies.T.reindex(done).T.fillna(0)
  Set.drop(columns=['AssetNo'], inplace = True)
  Set = pd.concat([Set, dummies], axis=1)

  FEATURES_SHAPE =  Set.shape[1]
  print(Set.shape)
  print(FEATURES_SHAPE)
  #Forming examples
  SEQ_LEN = 7
  Set =  series_to_supervised(Set, SEQ_LEN-1, 0)
  Set[["returns", "Signal"]] = Ana[["returns", "Signal"]]
  Set = Set.dropna()
#   print(Set.shape)
# Set.head()
  Sets = Sets.append(Set)

# Sort dataframe by dates
Sets = Sets.sort_index()

In [0]:
#check dataframe for training
Sets.head()

,buzz(t-7),sentiment(t-7),optimism(t-7),joy(t-7),loveHate(t-7),trust(t-7),anger(t-7),conflict(t-7),fear(t-7),gloom(t-7),stress(t-7),surprise(t-7),timeUrgency(t-7),uncertainty(t-7),violence(t-7),emotionVsFact(t-7),marketRisk(t-7),longShort(t-7),longShortForecast(t-7),priceDirection(t-7),priceForecast(t-7),volatility(t-7),analystRating(t-7),debtDefault(t-7),dividends(t-7),innovation(t-7),earningsForecast(t-7),fundamentalStrength(t-7),laborDispute(t-7),layoffs(t-7),litigation(t-7),managementChange(t-7),managementTrust(t-7),mergers(t-7),cyberCrime(t-7),r1(t-7),r2(t-7),r3(t-7),r4(t-7),buzz_avg30_diff90_avg(t-7),buzz_avg90_diff180_avg(t-7),sentiment_avg30_diff90_avg(t-7),sentiment_avg90_diff180_avg(t-7),optimism_avg30_diff90_avg(t-7),optimism_avg90_diff180_avg(t-7),joy_avg30_diff90_avg(t-7),joy_avg90_diff180_avg(t-7),loveHate_avg30_diff90_avg(t-7),loveHate_avg90_diff180_avg(t-7),trust_avg30_diff90_avg(t-7),trust_avg90_diff180_avg(t-7),anger_avg30_diff90_avg(t-7),anger_avg90_diff180_avg(t-7),conflict_avg30_diff90_avg(t-7),conflict_avg90_diff180_avg(t-7),fear_avg30_diff90_avg(t-7),fear_avg90_diff180_avg(t-7),gloom_avg30_diff90_avg(t-7),gloom_avg90_diff180_avg(t-7),stress_avg30_diff90_avg(t-7),stress_avg90_diff180_avg(t-7),surprise_avg30_diff90_avg(t-7),surprise_avg90_diff180_avg(t-7),timeUrgency_avg30_diff90_avg(t-7),timeUrgency_avg90_diff180_avg(t-7),uncertainty_avg30_diff90_avg(t-7),uncertainty_avg90_diff180_avg(t-7),violence_avg30_diff90_avg(t-7),violence_avg90_diff180_avg(t-7),emotionVsFact_avg30_diff90_avg(t-7),emotionVsFact_avg90_diff180_avg(t-7),marketRisk_avg30_diff90_avg(t-7),marketRisk_avg90_diff180_avg(t-7),longShort_avg30_diff90_avg(t-7),longShort_avg90_diff180_avg(t-7),longShortForecast_avg30_diff90_avg(t-7),longShortForecast_avg90_diff180_avg(t-7),priceDirection_avg30_diff90_avg(t-7),priceDirection_avg90_diff180_avg(t-7),priceForecast_avg30_diff90_avg(t-7),priceForecast_avg90_diff180_avg(t-7),volatility_avg30_diff90_avg(t-7),volatility_avg90_diff180_avg(t-7),analystRating_avg30_diff90_avg(t-7),analystRating_avg90_diff180_avg(t-7),debtDefault_avg30_diff90_avg(t-7),debtDefault_avg90_diff180_avg(t-7),dividends_avg30_diff90_avg(t-7),dividends_avg90_diff180_avg(t-7),innovation_avg30_diff90_avg(t-7),innovation_avg90_diff180_avg(t-7),earningsForecast_avg30_diff90_avg(t-7),earningsForecast_avg90_diff180_avg(t-7),fundamentalStrength_avg30_diff90_avg(t-7),fundamentalStrength_avg90_diff180_avg(t-7),laborDispute_avg30_diff90_avg(t-7),laborDispute_avg90_diff180_avg(t-7),layoffs_avg30_diff90_avg(t-7),layoffs_avg90_diff180_avg(t-7),litigation_avg30_diff90_avg(t-7),litigation_avg90_diff180_avg(t-7),managementChange_avg30_diff90_avg(t-7),managementChange_avg90_diff180_avg(t-7),managementTrust_avg30_diff90_avg(t-7),managementTrust_avg90_diff180_avg(t-7),mergers_avg30_diff90_avg(t-7),mergers_avg90_diff180_avg(t-7),cyberCrime_avg30_diff90_avg(t-7),cyberCrime_avg90_diff180_avg(t-7),0(t-7),1(t-7),2(t-7),3(t-7),4(t-7),5(t-7),6(t-7),7(t-7),8(t-7),9(t-7),buzz(t-6),sentiment(t-6),optimism(t-6),joy(t-6),loveHate(t-6),trust(t-6),anger(t-6),conflict(t-6),fear(t-6),gloom(t-6),stress(t-6),surprise(t-6),timeUrgency(t-6),uncertainty(t-6),violence(t-6),emotionVsFact(t-6),marketRisk(t-6),longShort(t-6),longShortForecast(t-6),priceDirection(t-6),priceForecast(t-6),volatility(t-6),analystRating(t-6),debtDefault(t-6),dividends(t-6),innovation(t-6),earningsForecast(t-6),fundamentalStrength(t-6),laborDispute(t-6),layoffs(t-6),litigation(t-6),managementChange(t-6),managementTrust(t-6),mergers(t-6),cyberCrime(t-6),r1(t-6),r2(t-6),r3(t-6),r4(t-6),buzz_avg30_diff90_avg(t-6),buzz_avg90_diff180_avg(t-6),sentiment_avg30_diff90_avg(t-6),sentiment_avg90_diff180_avg(t-6),optimism_avg30_diff90_avg(t-6),optimism_avg90_diff180_avg(t-6),joy_avg30_diff90_avg(t-6),joy_avg90_diff180_avg(t-6),loveHate_avg30_diff90_avg(t-6),loveHate_avg90_diff180_avg(t-6),trust_avg30_diff90_avg(t-6),trust_avg90_diff180_avg(t-6),anger_avg30_diff90_avg(t-6),anger_avg90_diff180_avg(t-6),conflict_av

In [0]:
## Only to use when removing model 
!rm best.h5

In [0]:
# Define matthews_correlation function to calculate MCC
def matthews_correlation(y_true, y_pred):
  y_pred_pos = K.round(K.clip(y_pred, 0, 1))
  y_pred_neg = 1 - y_pred_pos

  y_pos = K.round(K.clip(y_true, 0, 1))
  y_neg = 1 - y_pos

  tp = K.sum(y_pos * y_pred_pos)
  tn = K.sum(y_neg * y_pred_neg)

  fp = K.sum(y_neg * y_pred_pos)
  fn = K.sum(y_pos * y_pred_neg)

  numerator = (tp * tn - fp * fn)
  denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

  return numerator / (denominator + K.epsilon())

# LSTM helper function to reset weights of LSTM
def reset_weights(model):
    session = K.get_session()
    for layer in model.layers: 
        if isinstance(layer, keras.engine.network.Network):
            reset_weights(layer)
            continue
        for v in layer.__dict__.values():
            if hasattr(v, 'initializer'):
                v.initializer.run(session=session)

In [0]:
# set up dataframe tradeTable for training
tradeTable= Sets.copy()

# initialise "predictSignal" as all 0
tradeTable["predictSignal"] = 0

# initialise "trainingpredictSignal" as all 0
tradeTable["trainingpredictSignal"] = 0

# add 10 columns for at most 10 ensemble models results
tradeTable = pd.concat([tradeTable, pd.DataFrame(columns = ["0","1", "2","3","4","5","6","7","8","9"])], axis =1)
tradeTable[["0","1", "2","3","4","5","6","7","8","9"]] = [0,0,0,0,0,0,0,0,0,0]

In [0]:
start = time. time()

# roll is how many times u want to roll forward the train validate testing
# You can use roll 5 to automatically test 5 testing periods. Howver, model training seems to degrade when rolled too many times. 
# Hence, just use roll=1 and reset the the start date in the next line to 5 consecutive start test dates and run this code separately 5 times.
# The results will be appended to the tradeTable dataframe above

#Define Test Start Date
daterange = pd.date_range(start='1/1/2018', periods=roll , freq="3MS")
print(daterange)

EFF_TEST_START = ""
EFF_TEST_END = ""
EFF_TRAIN_START = ""
EFF_TRAIN_END = ""
TestStartList = []
TrainStartList = []
AccuracyList = []
num = 0 
#roll number count
rollno = 0

for single_date in daterange:
  
  rollno +=1
  testyhat = []
  trainyhat = []
  valyhat  = []
  num += 1
  localAcc =[] 
  
  #Number of days for training
  TrainVal = 61
  
  #Number of days for training
  TrainValEnd = 0
  
  #Number of ensembles model in ensemble periods
  ENSEMBLE = 3
  
  #number of months for the length of each test period
  testper = 3
  
  #Record Test Start Date
  TEST_START = (single_date).strftime("%Y-%m-%d")
  TEST_END = (single_date + relativedelta(months=+testper)- timedelta(days=1)).strftime("%Y-%m-%d")
  TestStartList += [np.array([TEST_START, TEST_END])]
  
  #Start Ensemble
  for i in range(0, ENSEMBLE):
    
    print("roll"  + str(rollno))
    print("Ensembel Model" + str(i))
    print(TrainVal)
    print(TrainValEnd)
    #Define Train period
    TRAIN_START = (single_date - timedelta(days=2) - relativedelta(days= TrainVal)).strftime("%Y-%m-%d")
    TRAIN_END = (single_date - timedelta(days=2) - relativedelta(days= TrainValEnd)).strftime("%Y-%m-%d")
    if i <= 0:
      increment = 61
    if i > 0:
      increment = 122
    TrainValEnd  += increment
    TrainVal += increment
    TrainStartList += [np.array([TRAIN_START, TRAIN_END])]

    print(TRAIN_START)
    print(TRAIN_END)
    print(TEST_START)
    print(TEST_END)
    
    #Record Effective Train Start, Train End, Test start, Test End dates
    if num == 1:
      EFF_TEST_START = TEST_START
      EFF_TRAIN_START = TRAIN_START
    if num == len(daterange):
      EFF_TEST_END = TEST_END
      EFF_TRAIN_END = TRAIN_END

    print(tradeTable.columns)
    #Set first x and and last x variable and y variable
    FIRSTTRAINVAR = tradeTable.columns[0]
    print(FIRSTTRAINVAR)
    LASTTRAINVARIABLE = tradeTable.columns[-15]
    print("lastTrainVar" +  str(LASTTRAINVARIABLE))
    YVAR= "Signal"

    #Specify LSTM input shape
    X_SHAPE1 = SEQ_LEN
    X_SHAPE2 = FEATURES_SHAPE
    
    #Set Xtrain, ytrain, Xval, yval, Xtest, ytest
    training = tradeTable.loc[TRAIN_START:TRAIN_END, FIRSTTRAINVAR:LASTTRAINVARIABLE]
    y = tradeTable.loc[TRAIN_START:TRAIN_END, YVAR]
    
    train_x , val_x, train_y, val_y = train_test_split(training, y, test_size=0.4, shuffle = True)
  #     print(train_y)
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_x = scaler.fit_transform(train_x)
    train_x = train_x.reshape(train_x.shape[0], X_SHAPE1, X_SHAPE2)
    print(train_x.shape)
    train_y = train_y.values.reshape((train_y.shape[0], 1))
    print(train_y.shape)
    val_x = scaler.transform(val_x)
    val_x = val_x.reshape(val_x.shape[0], X_SHAPE1, X_SHAPE2)
    print(val_x.shape)
    val_y = val_y.values.reshape((val_y.shape[0], 1))
    print(val_y.shape)

  #     trainingx = scaler.transform(training)
  #     trainingx = trainingx.reshape(trainingx.shape[0], X_SHAPE1, X_SHAPE2)

    #test_x
    test_x = tradeTable.loc[TEST_START:TEST_END, FIRSTTRAINVAR:LASTTRAINVARIABLE]
    # Normalised train_x to [0,1]
    test_x = scaler.transform(test_x)
    #reshape test_x for RNN input
    test_x = test_x.reshape(test_x.shape[0], X_SHAPE1, X_SHAPE2)
  #   print(test_x.shape)

    #test_y
    test_y = tradeTable.loc[TEST_START:TEST_END, YVAR]
    test_y = test_y.values.reshape((test_y.shape[0], 1))
    # test_y = yscaler.transform(test_y)

  #   print(test_y.shape)

    ## Specifying Batch Size
    n_batch = 4

    #Check train_y statistic
    trainStats = np.unique(train_y, return_counts=True)
    print(trainStats)
    
    #Compute class weight for model
    classweight = class_weight.compute_class_weight('balanced',
                                                   np.unique(train_y),
                                                   train_y.flatten())
    print(classweight)
    
    # Define LSTM model
    units = 8
    inputs = Input(shape=(train_x.shape[1], train_x.shape[2]))
    #   inpd = Dense(5)(inputs)
    #   inpd = Dense(30)(inpd)
    secondary = Bidirectional(LSTM(units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(inputs)
    secondary = BatchNormalization()(secondary)
    #     activations = LSTM(units)(secondary)
    activations = LSTM(units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2,)(secondary)
    activations = BatchNormalization()(activations)
    x = Dropout(0.3)(activations)

    attention1 = Dense(1, activation='relu')(x)
    attention2 = Flatten()(attention1)
    attention4 = Activation('softmax')(attention2)
    attention5 = RepeatVector(units)(attention4)
    attention6 = Permute([2, 1])(attention5)
    sent_representation = multiply([activations, attention6])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(units,))(sent_representation)

    probabilities = Dense(1, activation='sigmoid')(sent_representation)

    model = Model(inputs=inputs, outputs=probabilities)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
    
    # reset Model wewights
    reset_weights(model)
    
    # Fit model
    checkpoint = ModelCheckpoint("best.h5", monitor='val_binary_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max')
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='auto')
    
    history = model.fit(train_x, train_y, epochs=20,  batch_size=n_batch, callbacks=[checkpoint, early],
                        validation_data=(val_x, val_y), 
                        shuffle=False, verbose =  1,
                          class_weight= classweight
                          )

    # load the saved model
    if os.path.isfile("best.h5"):
      model.load_weights("best.h5")
    # Make predictions, yhat on test and validate set 
    yhat = model.predict(test_x, batch_size=n_batch)
    vyhat = model.predict(val_x, batch_size=n_batch)
    
    # print metrics for validate set
    print(confusion_matrix(val_y, vyhat>0.5))
    print(accuracy_score(val_y, vyhat>0.5))
    print(matthews_corrcoef(val_y, vyhat>0.5))
    
    # appending predictions for model in ensemeble
    testyhat += [yhat]
#     valyhat += [vyhat]

  nptestyhat = np.array(testyhat)
#   npvalyhat = np.array(valyhat)
  
  reshapeTestyhat = nptestyhat.reshape(nptestyhat.shape[0], nptestyhat.shape[1]).transpose()
#   reshapeValyhat = npvalyhat.reshape(npvalyhat.shape[0], npvalyhat.shape[1]).transpose()
  
#   print(confusion_matrix(val_y, np.mean(reshapeValyhat, axis =1)>0.5))
#   print(accuracy_score(val_y, np.mean(reshapeValyhat, axis =1)>0.5))
#   print(matthews_corrcoef(val_y, np.mean(reshapeValyhat, axis =1)>0.5))
  
  #record mean of all ensemble models prediction as the final prediction in tradeTable
  tradeTable.loc[TEST_START:TEST_END, "predictSignal"] = np.mean(reshapeTestyhat, axis =1) > 0.5
  
  #record each ensemble prediction in tradeTable
  tradeTable.loc[TEST_START:TEST_END, "0":str(ENSEMBLE-1)] = reshapeTestyhat

DatetimeIndex(['2018-01-01', '2018-04-01', '2018-07-01', '2018-10-01',
               '2019-01-01'],
              dtype='datetime64[ns]', freq='3MS')
roll1
Ensembel Model0
61
0
2017-10-30
2017-12-30
2018-01-01
2018-03-31
Index(['r1(t-6)', 'r2(t-6)', 'r3(t-6)', 'r4(t-6)', '0(t-6)', '1(t-6)',
       '2(t-6)', '3(t-6)', '4(t-6)', '5(t-6)',
       ...
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype='object', length=112)
r1(t-6)
lastTrainVar9(t-0)
(258, 7, 14)
(258, 1)
(172, 7, 14)
(172, 1)
(array([False,  True]), array([125, 133]))
[1.032      0.96992481]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 258 samples, validate on 172 samples
Epoch 1/20
258/258 [==============================] - 8s 32ms/step - loss: 0.8510 - binary_accuracy: 0.5000 - val_loss: 0.9496 - val_binar

In [0]:
#Save CSV file for the results
df = tradeTable.loc["2018-01-01":"2019-03-01","0(t-0)":].copy()
downloadfile = "roll" + str(roll)+ "ENS"+ str(ENSEMBLE)+ "Train3X"+"TEST"+ str(testper)+"ACC"+ str(accuracy) + "AUC"+ str(auc)+ "MCC" + str(mcc)+ str(TRMI)+ ".csv"
df.to_csv(downloadfile)
files.download(downloadfile)

In [0]:
# Make directory for saving images
!rm -r images
!mkdir images

# define confusion matrix function
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

#     plt.figure(figsize= (10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
#     plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=30)
    plt.yticks(tick_marks, classes, fontsize=30)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="black" if cm[i, j] > thresh else "black", fontsize=20)

    plt.ylabel('True label', fontsize=30)
    plt.xlabel('Predicted label', fontsize=30)
    plt.tight_layout()

# Define helper function to calculate confusion matix and other metrics  
def plotConfusion(df, start, end, string, col = "predictedweight"):
  real = df["labelledweight"]
  pred = df[col]
  cnf_matrix = confusion_matrix(real[start:end], pred[start:end])
  accuracy = np.around(accuracy_score(real[start:end], pred[start:end]),2)
  auc = np.around(areauc(real[start:end], pred[start:end]), 2)
  mcc =  np.around(matthews_corrcoef(real[start:end], pred[start:end]), 2)
  plot_confusion_matrix(cnf_matrix, classes=['Sell', 'buy'],
                        title=string + ' Accuracy: ' +str(accuracy)
                        + ', AUC:' + str(auc)
                        + ', MCC:' + str(mcc))
  
  return accuracy, auc, mcc
  
  
# Define Path  
from pathlib import Path
PATH = Path('images')

# copy TradeTable over for printing results
df = tradeTable.loc[EFF_TEST_START:EFF_TEST_END,:].copy()
# turning signals into numerics
df['predictedweight'] = np.where(df['predictSignal'] == True, 1, -1)
df['labelledweight'] = np.where(df['Signal'] == True, 1, -1)


# print for each stock the PNL and confusion matrix
for dum in range(10):
  filename = read[dum]
  print(filename)
  df.loc[df[str(dum)+ "(t-0)"] ==1, "predictedreturns"] = df.loc[df[str(dum)+ "(t-0)"] ==1, "returns"]*df.loc[df[str(dum)+ "(t-0)"] ==1, "predictedweight"].shift(1)
  dl = df.loc[df[str(dum)+ "(t-0)"] ==1, :]
  dl["PNL"] = (1+dl['predictedreturns']).cumprod()
  num = 0
  fig, ax = plt.subplots(figsize=(15,7))
  cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
  print(TestStartList)
  print(AccuracyList)
  # print PNL for each roling window a separate color
  for i in TestStartList:
    num +=1
    dl.loc[i[0]:i[1], "PNL"].plot(ax= ax, color="C{}".format(num% 10), fontsize=20)
  plt.show()
  fig.savefig(PATH/(str(filename) +'PNL.png'))
  
  # plotting Confusion Matrix
  f = plt.figure(figsize=(15,7))
  ax = f.add_subplot(131)
  plotConfusion(dl, EFF_TEST_START, EFF_TEST_END, "")
  f.savefig(PATH/(str(filename) +'CM.png'))
  plt.show()

# plotting Confusion Matrix for all the stocks
f = plt.figure(figsize=(15,7))
ax = f.add_subplot(131)
accuracy, auc, mcc = plotConfusion(df, EFF_TEST_START, EFF_TEST_END, "")

# plotting PNL for all the stocks
df["PNL"] = (1+df['predictedreturns']*0.1).cumprod()
df.head()
num = 0
fig, ax = plt.subplots(figsize=(15,7))
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
df.loc[EFF_TEST_START: EFF_TEST_END, "PNL"].plot(ax= ax, color="C{}".format(num% 10), fontsize=20)
plt.show()